In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [2]:
iris = datasets.load_iris()


In [3]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [4]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [5]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [6]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

In [7]:
def gain_calculator(df, y):
    current_entropy=0
    for class_ in set(y):
        current_entropy-=(sum(y==class_)/len(y))*np.log10(sum(y==class_)/len(y))
    ###############################
    weighted_sum_of_node_entropies=0
    split_info=0
    for class_ in set(df):
        current_y_values=y[df==class_]
        entropy_of_perticular_node=0
        for i in set(current_y_values):
            entropy_of_perticular_node-=(sum(current_y_values==i)/len(current_y_values))*np.log10(sum(current_y_values==i)/len(current_y_values))
        weighted_sum_of_node_entropies+=entropy_of_perticular_node*(len(current_y_values)/len(y))
        split_info-=(len(current_y_values)/len(y))*(np.log10(len(current_y_values)/len(y)))
    info_gain=current_entropy-weighted_sum_of_node_entropies
    gain_ratio=info_gain/split_info
    ###############################
    return current_entropy, gain_ratio
    
    
    
def build_tree(df, y, unused_features):
    #base case
    if (len(unused_features)==0) or (len(set(y))==1):
    # 1. unused is empty
    # 2. y contains only one distinct value
        return
    best_feature = ""
    final_gain=-1
    final_entropy=0
    for f in unused_features:
        current_entropy, gain_ratio=gain_calculator(df[f], y)
        if gain_ratio>final_gain:
            final_gain=gain_ratio
            best_feature=f
            final_entropy=current_entropy
    # here you should know the best feature
    # print it out
    for i in set(y):
        print("Count of", i, "=", sum(y==i))
    print("Current Entropy is =", final_entropy)
    if final_entropy==0:
        print('Reached leaf node')
    else:
        print("Splitting on feature", best_feature, "with gain ratio", final_gain)
    print()
    # remove best feature from unused features
    unused_features.remove(best_feature)
    # loop over possible values of best feature
    for possible_value in set(df[best_feature]):
        build_tree(df.loc[df[best_feature]==possible_value], y[df[best_feature]==possible_value], unused_features)
    # call build tree recursively

In [10]:
y = (iris.target)
print(y)
unused_features = set(df.columns)
print(unused_features)
build_tree(df, y, unused_features)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
{'pw_labeled', 'sw_labeled', 'sl_labeled', 'pl_labeled'}
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 0.4771212547196624
Splitting on feature pw_labeled with gain ratio 0.6996382036222089

Count of 1 = 40
Count of 2 = 16
Current Entropy is = 0.2598251810131059
Splitting on feature pl_labeled with gain ratio 0.43340994956210654

Count of 1 = 39
Count of 2 = 8
Current Entropy is = 0.1981353138938235
Splitting on feature sl_labeled with gain ratio 0.12674503775809334

Count of 1 = 23
Count of 2 = 7
Current Entropy is = 0.23594037110284793
Splitting on feature sw_labeled with gain ratio 0.07092036405148883

